In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


In [2]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 100)

In [3]:
#REDUCE MEMORY USAGE
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
train_df = pd.read_csv(r"E:\Data Science\Kaggle Competitions dataset/train.csv")

In [5]:
train_df = reduce_mem_usage(train_df)

Mem. usage decreased to 289.19 Mb (53.1% reduction)


In [6]:
y = train_df["meter_reading"]

In [7]:
import gc
del train_df
gc.collect()

0

In [8]:
#final_train = pd.read_csv(r"E:\Data Science\Kaggle Competitions dataset\Ashrae/final_train_NN.csv")

In [9]:
#from sklearn.preprocessing import MinMaxScaler

#scaler = MinMaxScaler()
#scaler.fit(final_train)
#normal = scaler.transform(final_train)

In [10]:
#normal.shape

In [11]:
#normal_temp = pd.DataFrame(normal)

In [12]:
#normal_temp.to_csv(r"E:\Data Science\Kaggle Competitions dataset\Ashrae\normal_final_train_NN.csv")

In [13]:
#del final_train
#del normal
#gc.collect()

In [14]:
#final_test = pd.read_csv(r"E:\Data Science\Kaggle Competitions dataset\Ashrae/final_test_NN1.csv")

In [15]:
#final_test.shape

In [16]:
#test_norm1 = final_test.iloc[:20000000,:]

In [17]:
#test_norm1 = reduce_mem_usage(test_norm1)

In [18]:
#test_norm1 = scaler.transform(test_norm1)

In [19]:
#test_norm1 = pd.DataFrame(test_norm1)

In [20]:
#test_norm2 = final_test.iloc[20000000:,:]

In [21]:
#test_norm2 = reduce_mem_usage(test_norm2)

In [22]:
#test_norm2 = scaler.transform(test_norm2)

In [23]:
#test_norm2 = pd.DataFrame(test_norm2)

In [24]:
#del final_test
#gc.collect()

In [25]:
#test_normal = pd.concat([test_norm1,test_norm2])

In [26]:
#del test_norm1
#del test_norm2
#gc.collect()

In [27]:
#normal_test = pd.DataFrame(test_normal)

In [28]:
#normal_test.to_csv(r"E:\Data Science\Kaggle Competitions dataset\Ashrae\normal_final_test_NN1.csv")

In [29]:
#final_train.head()

In [30]:
normal = pd.read_csv(r"E:\Data Science\Kaggle Competitions dataset\Ashrae\normal_final_train_NN.csv",index_col=False)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(normal, y, test_size=0.001, random_state=42)

In [32]:
del normal
gc.collect()

0

In [33]:
import tensorflow as tf
from tensorflow import keras

In [51]:
from tensorflow.keras.regularizers import l2 as reg2,l1 as reg1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,BatchNormalization,Dropout


model4 = Sequential()
model4.add(Dense(121,activation=tf.nn.relu ,kernel_initializer='lecun_uniform'))
model4.add(BatchNormalization())
model4.add(Dense(500,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = reg2(0.01)))
model4.add(BatchNormalization())
model4.add(Dense(500,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = reg2(0.01)))
model4.add(BatchNormalization())
model4.add(Dense(200,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = reg2(0.01)))
model4.add(Dense(1,activation="linear",kernel_initializer='lecun_uniform'))


In [52]:
adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [53]:
model4.compile(optimizer=adam, loss='mean_squared_error',metrics=['mean_squared_error'])

In [54]:
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

In [55]:
# 25 epochs LR of 0.001 batch 10K val MSE is 1357837824
# Reducing LR to 0.0001 30 epoch MSE 1077661952 
# 

In [56]:
history4 = model4.fit(X_train.values, y_train.values, epochs=40,batch_size = 10000,validation_split = 0.1)

Train on 18176294 samples, validate on 2019589 samples
Epoch 1/40
18176294/18176294 [==============================] - 220s 12us/sample - loss: 23050131155.9533 - mean_squared_error: 23049648128.0000 - val_loss: 31847995297.1584 - val_mean_squared_error: 31847958528.0000
Epoch 2/40
18176294/18176294 [==============================] - 102s 6us/sample - loss: 21873798718.3911 - mean_squared_error: 21873414144.0000 - val_loss: 205528472261.9185 - val_mean_squared_error: 205527777280.0000
Epoch 3/40
18176294/18176294 [==============================] - 102s 6us/sample - loss: 20218986563.7700 - mean_squared_error: 20218718208.0000 - val_loss: 194523960516.9345 - val_mean_squared_error: 194521792512.0000
Epoch 4/40
18176294/18176294 [==============================] - 103s 6us/sample - loss: 18363762500.5603 - mean_squared_error: 18363494400.0000 - val_loss: 38200848583007.8125 - val_mean_squared_error: 38200697421824.0000
Epoch 5/40
18176294/18176294 [==============================] - 102s 6

Epoch 38/40
18176294/18176294 [==============================] - 101s 6us/sample - loss: 2494132731.2301 - mean_squared_error: 2494134784.0000 - val_loss: 1739775598992.7051 - val_mean_squared_error: 1739758108672.0000
Epoch 39/40
18176294/18176294 [==============================] - 101s 6us/sample - loss: 1883045895.5531 - mean_squared_error: 1883033856.0000 - val_loss: 238078939433.2529 - val_mean_squared_error: 238078918656.0000
Epoch 40/40
18176294/18176294 [==============================] - 102s 6us/sample - loss: 1886073951.0449 - mean_squared_error: 1886062208.0000 - val_loss: 201275513727.8962 - val_mean_squared_error: 201275637760.0000


In [57]:
model_json = model4.to_json()
with open("modelNN_1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model4.save_weights("modelNN_1.h5")
print("Saved model to disk")

Saved model to disk


In [41]:
del model4
del history4
gc.collect()


189777

In [42]:
model = Sequential()
model.add(Dense(121,activation=tf.nn.relu ,kernel_initializer='lecun_uniform'))
model.add(BatchNormalization())
model.add(Dense(500,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = reg2(0.01)))
model.add(BatchNormalization())
model.add(Dense(500,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = reg2(0.01)))
model.add(BatchNormalization())
model.add(Dense(500,activation=tf.nn.tanh ,kernel_initializer='lecun_uniform',kernel_regularizer = reg2(0.01)))
model.add(BatchNormalization())
model.add(Dense(400,activation=tf.nn.tanh ,kernel_initializer='lecun_uniform'))
model.add(BatchNormalization())
model.add(Dense(200,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = reg2(0.01)))
model.add(BatchNormalization())
model.add(Dense(100,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = reg2(0.01)))
model.add(Dense(1,activation="linear" ,kernel_initializer='lecun_uniform'))

In [43]:
adam = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [44]:
model.compile(optimizer=adam, loss='mean_squared_error',metrics=['mean_squared_error'])

In [45]:
history = model.fit(X_train.values, y_train.values, epochs=100,batch_size = 10000,validation_split = 0.1)

Train on 18176294 samples, validate on 2019589 samples
Epoch 1/100
18176294/18176294 [==============================] - 168s 9us/sample - loss: 23361763625.1093 - mean_squared_error: 23361734656.0000 - val_loss: 24224783322.3795 - val_mean_squared_error: 24224774144.0000
Epoch 2/100
18176294/18176294 [==============================] - 48s 3us/sample - loss: 23361650718.0943 - mean_squared_error: 23361634304.0000 - val_loss: 24224641082.1729 - val_mean_squared_error: 24224647168.0000
Epoch 3/100
18176294/18176294 [==============================] - 48s 3us/sample - loss: 23361503511.8087 - mean_squared_error: 23361472512.0000 - val_loss: 24224556927.3816 - val_mean_squared_error: 24224552960.0000
Epoch 4/100
18176294/18176294 [==============================] - 48s 3us/sample - loss: 23361322156.6035 - mean_squared_error: 23361304576.0000 - val_loss: 24224256982.5199 - val_mean_squared_error: 24224256000.0000
Epoch 5/100
18176294/18176294 [==============================] - 49s 3us/sample 

18176294/18176294 [==============================] - 48s 3us/sample - loss: 23352487197.4370 - mean_squared_error: 23352504320.0000 - val_loss: 24214594094.1509 - val_mean_squared_error: 24214587392.0000
Epoch 39/100
18176294/18176294 [==============================] - 48s 3us/sample - loss: 23352452405.2371 - mean_squared_error: 23352446976.0000 - val_loss: 24214875876.2914 - val_mean_squared_error: 24214865920.0000
Epoch 40/100
18176294/18176294 [==============================] - 48s 3us/sample - loss: 23352346611.9849 - mean_squared_error: 23352358912.0000 - val_loss: 24215055910.8439 - val_mean_squared_error: 24215056384.0000
Epoch 41/100
18176294/18176294 [==============================] - 48s 3us/sample - loss: 23352393520.5937 - mean_squared_error: 23352410112.0000 - val_loss: 24215367123.9964 - val_mean_squared_error: 24215365632.0000
Epoch 42/100
18176294/18176294 [==============================] - 48s 3us/sample - loss: 23352399946.5147 - mean_squared_error: 23352379392.0000 

Epoch 76/100
18176294/18176294 [==============================] - 48s 3us/sample - loss: 23351863488.2044 - mean_squared_error: 23351861248.0000 - val_loss: 24213866718.5018 - val_mean_squared_error: 24213872640.0000
Epoch 77/100
18176294/18176294 [==============================] - 48s 3us/sample - loss: 23351831238.1298 - mean_squared_error: 23351838720.0000 - val_loss: 24214204881.5972 - val_mean_squared_error: 24214198272.0000
Epoch 78/100
18176294/18176294 [==============================] - 48s 3us/sample - loss: 23352102131.9044 - mean_squared_error: 23352104960.0000 - val_loss: 24215812268.6097 - val_mean_squared_error: 24215822336.0000
Epoch 79/100
18176294/18176294 [==============================] - 48s 3us/sample - loss: 23351987492.8947 - mean_squared_error: 23351977984.0000 - val_loss: 24213863929.2889 - val_mean_squared_error: 24213852160.0000
Epoch 80/100
18176294/18176294 [==============================] - 48s 3us/sample - loss: 23351972583.2602 - mean_squared_error: 2335

In [47]:
model_json = model.to_json()
with open("modelDeepNN_1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("modelDeepNN_1.h5")
print("Saved model to disk")

Saved model to disk
